In [202]:
from hjst_model.config import DatasetKVSConfig, LayerModelConfig, HierarchicalModelConfig
import numpy as np
import scipy
import nmslib
import pandas as pd
import os
import re
import sentencepiece as spm

import sys 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from collections import Counter
from jstatutree import etypes
from pathlib import Path
try:
    import faiss
except:
    print('WARNING: cannot use faiss library')

#カラム内の文字数。デフォルトは50だった
pd.set_option("display.max_colwidth", 800)
pd.set_option("display.width", 3)

#行数
pd.set_option("display.max_rows", 101)

def get_code(tag, target_level):
    j=0
    tag_parts = Path(tag).parts
    for p in tag_parts:
        if target_level in p:
            break
        j += 1
    return '/'.join(tag_parts[:j+1])

def coloring(threshold, val, qcols, except_cols=[]):
    ret = ['background-color: white'] * len(val)
    for i, v in enumerate(val):
        if i in except_cols:
            continue
        if not isinstance(v, str) and v < threshold:
            for qcol in qcols:
                ret[qcol] = 'background-color: yellow' 
            ret[i] = 'background-color: yellow'
            ret[i-1] = 'background-color: yellow'
    return ret

def get_caption(query):
    try:
        return testset.get_elem(query+"/{}Caption(1)".format(re.split("\(", os.path.split(query)[1])[0])).text
    except Exception as e:
        return ""

TypeError: list indices must be integers or slices, not tuple

In [4]:
TESTSET = "aichi_muni"
CONF_DIR = './configs/'
HMODEL_NAME = 'tfidf_lsi_tokai_base'

testset_conf = DatasetKVSConfig()
testset_conf.link(os.path.join(CONF_DIR, "dataset.conf"))
testset_conf.change_section(TESTSET)
testset = testset_conf.prepare_dataset(False)

# prepare layer config instance
layerconf = LayerModelConfig()
layerconf.link(os.path.join(CONF_DIR, "layer.conf"))

model_conf = HierarchicalModelConfig()
model_conf.link(os.path.join(CONF_DIR, HMODEL_NAME+'.conf'), create_if_missing=True)
hmodel = model_conf.model_generate()
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
ltag_gen = (ltag for ltag in list(testset))
#hmodel.topk_comptable(testset, '23/230006/0001/Law(1)', 2, 0.2, 'Sentence')

The attribute skipped.
Load kvsdict from the path below:
/home/jovyan/hjst_model/results/hjst/models/tfidf_lsi300_tokai_mecab/Law.lmodel/vecs.ldb
set layer: TfidfLayer 0.0
The attribute skipped.
Load kvsdict from the path below:
/home/jovyan/hjst_model/results/hjst/models/tfidf_lsi300_tokai_mecab/Article.lmodel/vecs.ldb
set layer: TfidfLayer 0.0
The attribute skipped.
Load kvsdict from the path below:
/home/jovyan/hjst_model/results/hjst/models/tfidf_lsi300_tokai_mecab/Sentence.lmodel/vecs.ldb
set layer: TfidfLayer 0.0


In [3]:
hmodel.layers['Sentence'] = layerconf.load_model('swem_ave_gv_tokai_spm_8000-Sentence')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/hjst_model/results/hjst/models/swem_ave_gv_tokai_spm_8000/Sentence.lmodel/layer_instance.pkl'

In [17]:
# データセット変更用
del testset
testset_conf.change_section("tokai")
testset = testset_conf.prepare_dataset(False)
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
ltag_gen = (ltag for ltag in list(testset))

In [4]:
len(list(hmodel.layers['Law'].vecs.keys()))

62857

In [4]:
# Lawレベルに対してインデックス作成
law_tags = list(testset)
_, law_index = hmodel.create_index('Law', law_tags)

In [4]:
# テストレイヤーの読み込み
test_sentence_layers = {l:{'layer': layerconf.load_model(l+'-Sentence'), 'threshold': th} for (l, th) in [
    ('swem_average_tokai_mecab', 0.2),
    ('swem_ave_ft_tokai_spm_8000', 0.2),
    #('swem_ave_ft_tokai_spm_16000', 0.2),
    ('swem_ave_gv_tokai_spm_8000', 0.2),
    #('swem_ave_gv_tokai_spm_16000', 0.2),
    #("swem_ave_ft_tokai_mecab", 0.2),
    #('swem_concat_tokai_mecab', 0.2),
    #('swem_max_tokai_mecab',0.01),
    ('swem_average_tokai_spm_8000',0.2),
    #('swem_concat_tokai_spm_8000',0.2),
    #('swem_average_tokai_spm_16000',0.2),
    #('swem_concat_tokai_spm_16000',0.2),
    #('tf_tokai_spm_8000',0.5),
    #('tfidf_tokai_spm_8000',0.5)
]}
test_sentence_layers["swem_ave_ft_tokai_mecab"] = {"layer": hmodel.layers["Sentence"], 'threshold': 0.2}

In [5]:
# 各テストレイヤーに対してSentenceインデックス作成
testset.set_iterator_mode(level='Sentence', tag=True, sentence=False)
tags = list(testset)
for k in test_sentence_layers.keys():
    print(k)
    if 'index' in test_sentence_layers[k]:
        continue
    test_sentence_layers[k]['vectors'] = np.matrix([test_sentence_layers[k]['layer'][tag] for tag in tags])
    index = nmslib.init(method='hnsw', space='cosinesimil')
    index.addDataPointBatch(test_sentence_layers[k]['vectors'])
    index.createIndex({'post': 2}, print_progress=True)
    test_sentence_layers[k]['index'] = index
ltag_gen = (ltag for ltag in list(testset))

swem_average_tokai_mecab
swem_ave_ft_tokai_spm_8000
swem_ave_gv_tokai_spm_8000
swem_average_tokai_spm_8000
swem_ave_ft_tokai_mecab


In [9]:
# キーワード検索
keyword = "暴力団"
lt = None
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
for ltag in testset:
    if keyword in str(testset.get_elem(ltag)):
        lt = ltag
        print(lt, str(testset.get_elem(lt)))

23/230006/1797/Law(1) 愛知県公安委員会の権限に属する暴力団員による不当な行為の防止等に関する法律に規定する事務の一部を警察本部長等に委任する規則
23/230006/1813/Law(1) 愛知県暴力団排除条例
23/230006/1814/Law(1) 愛知県暴力団排除条例施行規則
23/231002/0999/Law(1) 名古屋市暴力団排除条例
23/232017/0086/Law(1) 豊橋市暴力団排除条例
23/232025/0692/Law(1) 岡崎市暴力団排除条例
23/232033/0700/Law(1) 一宮市暴力団等の排除に関する条例
23/232041/0562/Law(1) 瀬戸市暴力団排除条例
23/232050/0493/Law(1) 半田市暴力団排除条例
23/232068/0614/Law(1) 春日井市暴力団排除条例
23/232076/0498/Law(1) 豊川市暴力団排除条例
23/232084/0069/Law(1) 津島市暴力団排除条例
23/232092/0388/Law(1) 碧南市暴力団排除条例
23/232106/0325/Law(1) 刈谷市暴力団排除条例
23/232114/0797/Law(1) 豊田市暴力団排除条例
23/232122/0380/Law(1) 安城市暴力団排除条例
23/232131/0401/Law(1) 西尾市暴力団排除条例
23/232149/0555/Law(1) 蒲郡市暴力団排除条例
23/232157/0545/Law(1) 犬山市暴力団排除条例
23/232165/0610/Law(1) 常滑市暴力団排除条例
23/232173/0447/Law(1) 江南市暴力団排除条例
23/232190/0528/Law(1) 小牧市暴力団排除条例
23/232203/0089/Law(1) 稲沢市暴力団排除条例
23/232211/0595/Law(1) 新城市暴力団排除条例
23/232246/0460/Law(1) 知多市暴力団排除条例
23/232254/0273/Law(1) 知立市暴力団排除条例
23/232262/0345/Law(1) 尾張旭市暴力団排除条例
23/232271/0570/Law(1) 高浜市暴力団排除条例
23/232289/03

In [138]:
query = "24/245437/0420/Law(1)"
k = 100
root_threshold =  1000
threshold = 0.3
print("query:", testset.get_elem(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
tidx, distance = law_index.knnQuery(hmodel.layers[qlevel][query], k=k+1)
target_law_tags = [query]
for i, dist in zip(tidx, distance):
    if law_tags[i] == query:
        continue
    if root_threshold and dist > root_threshold:
        print("not enough distance", testset.get_elem(law_tags[i]), dist)
        k -= 1
        continue
    print("add target:", testset.get_elem(law_tags[i]), dist)
    target_law_tags.append(law_tags[i])

query: 紀北町農林業試作園条例


NameError: name 'law_index' is not defined

In [140]:
query = "24/245437/0420/Law(1)"
query_level = re.split('\(', os.path.split(query)[1])[0]
comp_level = 'Sentence'
query_k = 1000
comp_k = 10000
threshold = 0.2
print("query:", testset.get_elem(query))

query: 紀北町農林業試作園条例


In [18]:
testset.set_iterator_mode(level=comp_level, tag=True, sentence=False)
comp_level_tags = list(testset)
comp_level_matrix = np.array([hmodel.layers[comp_level][tag].astype(np.float32) for tag in comp_level_tags])
print(comp_level_matrix.shape)
print('using', len(comp_level_tags), comp_level+'s')

testset.set_iterator_mode(level=query_level, tag=True, sentence=False)
query_level_tags = list(testset)
query_level_matrix = np.matrix([hmodel.layers[query_level][tag].astype(np.float32) for tag in query_level_tags])
print(query_level_matrix.shape)
print('using', len(query_level_tags), query_level+'s')

(1920290, 300)
using 1920290 Sentences
(62857, 300)
using 62857 Laws


In [19]:
result_df = dict()

print('Construct index for', comp_level, 'level')
start = time.time()
comp_level_index = faiss.IndexFlatL2(comp_level_matrix.shape[1])   # build the index
comp_level_index.add(comp_level_matrix)                  # add vectors to the index
end = time.time()
print(comp_level, 'index preparation time: %f' % (end - start))

print('Construct index for', query_level, 'level')
start = time.time()
query_level_index = faiss.IndexFlatL2(query_level_matrix.shape[1])   # build the index
query_level_index.add(query_level_matrix)                  # add vectors to the index
end = time.time()
print(query_level, 'index preparation time: %f' % (end - start))

Construct index for Sentence level
Sentence index preparation time: 0.496176
Construct index for Law level
Law index preparation time: 0.016710


In [22]:
query_tags = [elem.code for elem in testset.get_elem(query).depth_first_iteration([getattr(etypes, comp_level)])]
query_matrix = np.matrix([hmodel.layers[comp_level][tag].astype(np.float32) for tag in query_tags])
start = time.time()
distance_matrix, idx_matrix = comp_level_index.search(query_matrix, comp_k)
end = time.time()
print('over all bottom up search time', round(end-start, 3), 'sec')
for i in range(idx_matrix.shape[0]):
    print(testset.get_elem(query_tags[i]))
    for j in range(5):
        print(j, testset.get_elem(comp_level_tags[idx_matrix[i][j]]), distance_matrix[i][j])
        print(testset.get_elem(comp_level_tags[idx_matrix[i][j]]).text)
    print()

over all bottom up search time 1.817 sec
愛知県暴力団排除条例第1章第1条第1項第1文
0 愛知県暴力団排除条例第1章第1条第1項第1文 5.9604645e-08
この条例は、暴力団の排除について、基本理念を定め、並びに県、事業者及び県民の責務を明らかにするとともに、県が実施する施策の基本となる事項等を定め、及び必要な規制を行うことにより、県、事業者及び県民が一体となって暴力団の排除を推進し、もって地域経済の健全な発展に寄与し、及び県民の安全で平穏な生活を確保することを目的とする。
1 飛島村暴力団排除条例第1条第1項第1文 0.024354517
この条例は、飛島村からの暴力団の排除について、基本理念を定め、並びに村、村民及び事業者の責務を明らかにするとともに、村が実施する施策の基本となる事項等を定めることにより、村、村民及び事業者が一体となって暴力団の排除を推進し、もって地域経済の健全な発展に寄与し、及び村民の安全で平穏な生活を確保することを目的とする。
2 蟹江町暴力団排除条例第1条第1項第1文 0.027037382
この条例は、蟹江町からの暴力団の排除について、基本理念を定め、並びに町、町民及び事業者の責務を明らかにするとともに、町が実施する施策の基本となる事項等を定めることにより、町、町民及び事業者が一体となって暴力団の排除を推進し、もって地域経済の健全な発展に寄与し、及び町民の安全で平穏な生活を確保することを目的とする。
3 幸田町暴力団排除条例第1条第1項第1文 0.027052581
この条例は、幸田町からの暴力団の排除について、基本理念を定め、並びに町、町民及び事業者の責務を明らかにするとともに、町が実施する施策の基本となる事項等を定めることにより、町、町民及び事業者が一体となって暴力団の排除を推進し、もって町民の安全で平穏な生活を確保し、及び地域経済の健全な発展に寄与することを目的とする。
4 扶桑町暴力団排除条例第1条第1項第1文 0.027114868
この条例は、扶桑町からの暴力団の排除について、基本理念を定め、並びに町、町民及び事業者の責務を明らかにするとともに、町が実施する施策の基本となる事項等を定めることにより、町、町民及び事業者が一体となって暴力団の排除を推進し、もって地域経済の健全な発展

3 稲沢市暴力団排除条例第7条第2項第1文 0.010108709
市長又は教育委員会は、公の施設の利用を許可した後において、当該公の施設が暴力団の活動に利用されることにより当該暴力団の利益になると認めるときは、当該公の施設の設置及び管理に関する事項を定めた条例の規定により、当該許可を取り消し、又は利用の中止を命ずることができるものとする。
4 名古屋市暴力団排除条例第7条第2項第1文 0.02115488
市長又は教育委員会は、公の施設の利用の許可をした後において、当該公の施設が暴力団の活動に利用されることにより当該暴力団の利益になると認めるときは、当該公の施設の設置及び管理に関する事項を定めた条例の規定にかかわらず、当該許可を取り消し、又は利用の中止を命ずることができるものとする。

愛知県暴力団排除条例第2章第10条第1項第1文
0 愛知県暴力団排除条例第2章第10条第1項第1文 2.9802322e-08
県は、推進センター等と連携し、県民及び事業者(以下「県民等」という。)が暴力団の排除に自主的に、かつ、相互に連携して取り組むことができるよう、県民等に対し、情報の提供その他の必要な支援を行うものとする。
1 阿久比町暴力団排除条例第8条第1項第1文 0.04048592
町は、県及び推進センター等と連携し、町民等が暴力団の排除に自主的に、かつ、相互に連携して取り組むことができるよう、町民等に対し、情報の提供その他の必要な支援を行うものとする。
2 武豊町暴力団排除条例第8条第1項第1文 0.04048592
町は、県及び推進センター等と連携し、町民等が暴力団の排除に自主的に、かつ、相互に連携して取り組むことができるよう、町民等に対し、情報の提供その他の必要な支援を行うものとする。
3 大口町暴力団排除条例第8条第1項第1文 0.07890281
町は、県及び暴追センター等と連携し、町民等が暴力団の排除のための活動に自主的に、かつ、相互に連携して取り組むことができるよう、町民等に対し、情報の提供その他の必要な支援を行うものとする。
4 東浦町暴力団排除条例第8条第1項第1文 0.08118409
町は、町民等が暴力団の排除に自主的に、かつ、相互に連携して取り組むことができるよう、町民等に対し、情報の提供その他の必要な支援を行うものとする。

愛知県暴力団排除

当該契約の相手方は、当該不動産を暴力団事務所の用に供してはならない旨
2 愛知県暴力団排除条例第4章第16条第4項第1文 0.11045188
第二項第二号に掲げる旨を定めた契約により不動産の譲渡等をした者は、当該不動産が暴力団事務所の用に供されていることが判明したときは、速やかに、当該契約を解除し、又は当該不動産の買戻しをするよう努めなければならない。
3 静岡県暴力団排除条例第6章第19条第3項第2号第1文 0.1431753
当該不動産が暴力団事務所の用に供されていることが判明したときは、当該譲渡等をした者は、当該契約を解除し、又は当該不動産の買戻しをすることができる旨
4 租税特別措置法に基づく優良宅地等の認定事務及び租税特別措置法施行令に基づく住宅用家屋の証明事務に関する規則第4条第3項第2号第1文 0.1475169
当該家屋の売買契約書、売渡証書等

愛知県暴力団排除条例第4章第16条第2項第2号第1文
0 愛知県暴力団排除条例第4章第16条第2項第2号第1文 -2.0861626e-07
当該不動産が暴力団事務所の用に供されていることが判明したときは、当該不動産の譲渡等をした者は、催告をすることなく当該契約を解除し、又は当該不動産の買戻しをすることができる旨
1 静岡県暴力団排除条例第6章第19条第3項第2号第1文 0.02212441
当該不動産が暴力団事務所の用に供されていることが判明したときは、当該譲渡等をした者は、当該契約を解除し、又は当該不動産の買戻しをすることができる旨
2 三重県動物の愛護及び管理に関する条例第9条第1項第3号第1文 0.09271604
当該犬に口輪をつけること。
3 愛知県暴力団排除条例第4章第16条第4項第1文 0.117251396
第二項第二号に掲げる旨を定めた契約により不動産の譲渡等をした者は、当該不動産が暴力団事務所の用に供されていることが判明したときは、速やかに、当該契約を解除し、又は当該不動産の買戻しをするよう努めなければならない。
4 稲沢市予算決算会計規則第3章第5節第48条第3項第3号第1文 0.12313272
当該債務が時効になつていないこと。

愛知県暴力団排除条例第4章第16条第3項第1文
0 愛知県暴力団排除条例第4章第16条第3項第1文 2.9802322e-08
不動産の譲渡等をしよ

3 東員町財務規則第10章第3節第283条第3項第1文 0.3200609
法第240条第4項に規定する債権については、この節の規定は適用しない。
4 松阪市会計規則第8章第102条第3項第1文 0.3200609
法第240条第4項に規定する債権については、この章の規定は適用しない。

愛知県暴力団排除条例第5章第18条第2項第2文
0 愛知県暴力団排除条例第5章第18条第2項第2文 1.7881393e-07
ただし、ある暴力団のものとして運営されていた暴力団事務所が、他の暴力団のものとして開設され、又は運営された場合は、この限りでない。
1 静岡県暴力団排除条例第3章第13条第2項第2文 0.072306216
ただし、ある暴力団のものとして運営されていたこれらの暴力団事務所が、この条例の施行又は適用の後に他の暴力団のものとして開設され、又は運営された場合は、この限りでない。
2 御嵩町ふるさとふれあい振興補助金交付規則第6条第1項第2号第1文 0.28809595
申請者の役員等が暴力団員(暴対法第2条第6号に規定する暴力団員をいう。以下同じ。)であるなど、暴力団がその経営又は運営に実質的に関与している場合
3 御嵩町街路灯設置等に係る補助金交付規則第6条第1項第2号第1文 0.28809595
申請者の役員等が暴力団員(暴対法第2条第6号に規定する暴力団員をいう。以下同じ。)であるなど、暴力団がその経営又は運営に実質的に関与している場合
4 牧之原市補助金等交付規則第3条第1項第5号第1文 0.29718727
暴力団員又は暴力団員等がその経営又は運営に実質的に関与しているとき。

愛知県暴力団排除条例第5章第19条第1項第1文
0 愛知県暴力団排除条例第5章第19条第1項第1文 2.9802322e-08
暴力団員は、正当な理由がある場合を除き、自己が活動の拠点とする暴力団事務所に青少年を立ち入らせてはならない。
1 伊豆の国市市営住宅管理条例第2章第9条第2項第3号第1文 0.14292237
新たに同居させようとする者が暴力団員である場合
2 掛川市営住宅管理条例第2章第12条第2項第3号第1文 0.15099558
同居させようとする者が暴力団員である場合
3 南知多町暴力団排除条例第3条第1項第1文 0.15386909
暴力団の排除は、暴力団が

特定接客業者は、特別区域における特定接客業の事業に関し、暴力団員に対し、顧客その他の者との紛争が発生した場合に用心棒の役務の提供を受ける目的で連絡をし、若しくは連絡を求め、又は面会をしてはならない。
3 愛知県暴力団排除条例第6章第23条第3項第1文 0.058858603
暴力団員は、特別区域における特定接客業の事業に関し、特定接客業者に対し、顧客その他の者との紛争が発生した場合に用心棒の役務の提供を受けさせる目的で連絡をし、若しくは連絡を求め、又は面会をしてはならない。
4 愛知県暴力団排除条例第6章第23条第1項第1文 0.06787968
暴力団員は、特別区域における特定接客業の事業に関し、特定接客業者に対し、その事業所における用心棒の役務の提供をしてはならない。

愛知県暴力団排除条例第6章第22条第3項第1文
0 愛知県暴力団排除条例第6章第22条第3項第1文 0.0
特定接客業者は、特別区域における特定接客業の事業に関し、暴力団員に対し、顧客その他の者との紛争が発生した場合に用心棒の役務の提供を受ける目的で連絡をし、若しくは連絡を求め、又は面会をしてはならない。
1 愛知県暴力団排除条例第6章第23条第3項第1文 0.01558587
暴力団員は、特別区域における特定接客業の事業に関し、特定接客業者に対し、顧客その他の者との紛争が発生した場合に用心棒の役務の提供を受けさせる目的で連絡をし、若しくは連絡を求め、又は面会をしてはならない。
2 愛知県暴力団排除条例第6章第22条第2項第1文 0.041647464
特定接客業者は、特別区域における特定接客業の事業に関し、暴力団員に対し、顧客その他の者との紛争が発生した場合に用心棒の役務の提供を受けることの対償として利益の供与をしてはならない。
3 愛知県暴力団排除条例第6章第23条第2項第1文 0.07245988
暴力団員は、特別区域における特定接客業の事業に関し、特定接客業者から、顧客その他の者との紛争が発生した場合に用心棒の役務の提供をすることの対償として利益の供与を受けてはならない。
4 愛知県暴力団排除条例第6章第23条第1項第1文 0.07979801
暴力団員は、特別区域における特定接客業の事業に関し、特定接客業者に対し、その事業所における用心棒の役務の提供をしてはならない。

愛知県暴力団排

4 鳥羽市民の環境と自然を守る条例第7章第68条第1項第1文 0.017953753
第34条及び第51条の規定による命令に違反した者は、3月以下の禁錮又は3万円以下の罰金に処する。

愛知県暴力団排除条例第9章第31条第1項第1文
0 愛知県暴力団排除条例第9章第31条第1項第1文 1.1920929e-07
法人(法人でない団体で代表者又は管理人の定めのあるものを含む。以下この項において同じ。)の代表者若しくは管理人又は法人若しくは人の代理人、使用人その他の従業者が、その法人又は人の業務に関し、前二条の違反行為をしたときは、行為者を罰するほか、その法人又は人に対しても、各本条の罰金刑を科する。
1 静岡県暴力団排除条例第10章第29条第1項第1文 0.03020525
法人(法人でない団体で代表者又は管理人の定めのあるものを含む。以下この項において同じ。)の代表者又は法人若しくは人の代理人、使用人その他の従業者が、その法人又は人の業務に関し、前条に規定する違反行為をしたときは、行為者を罰するほか、その法人又は人に対しても、同条の罰金刑を科する。
2 名古屋市定住促進住宅条例第34条第1項第1文 0.031130552
指定管理業務を行う法人(法人でない団体で代表者又は管理人の定めのあるものを含む。以下この条において同じ。)の代表者若しくは管理人又は法人若しくは人の代理人、使用人その他の従業者が、その法人又は人の業務に関し、前2条の違反行為をしたときは、その行為者を罰するほか、その法人又は人に対しても、各本条の罰金刑を科する。
3 津市個人情報保護条例第5章第56条第1項第1文 0.036643147
受託者又は指定管理者となった法人(法人でない団体で代表者又は管理人の定めのあるものを含む。以下この項において同じ。)の代表者若しくは管理人又は法人若しくは人の代理人、使用人その他の従業者が、その法人又は人の業務に関して、第53条又は第54条の違反行為をしたときは、行為者を罰するほか、その法人又は人に対しても、各本条の罰金刑を科する。
4 名張市個人情報保護条例第6章第37条第1項第1文 0.04678023
第11条第1項の委託を受けた法人(法人でない団体で代表者又は管理人の定めのあるものを含む。以下この条において同じ。)の代表者又は法人若しくは人の代理人、使用人そ

In [273]:
threshold = 0.2
sigmoid_alpha

def sigmoid_normalize(x):
    y = 1 / (1+np.exp(-x))
    return y/np.linalg.norm(y)

def softmax_normalize(x):
    exp_x = np.exp(x)
    y = exp_x / np.sum(np.exp(x), axis=0, keepdims=True)
    return y

def mean_normalize(x):
    return x/np.sum(x)

normalizer = mean_normalize

# 条文対応表作成特化型検索
query_matrix = np.matrix([hmodel.layers[query_level][query].astype(np.float32)])
print(query_matrix.shape)
start = time.time()
d, i = query_level_index.search(query_matrix, query_k)
end = time.time()
qsearch_result =  list(zip(i[0], d[0]))
print('search', len(qsearch_result), query_level+'s', 'from', len(query_tags), query_level+'s')
print('over all top search time', round(end-start, 3), 'sec')

query_tags = [elem.code for elem in testset.get_elem(query).depth_first_iteration([getattr(etypes, comp_level)])]
query_matrix = np.matrix([hmodel.layers[comp_level][tag].astype(np.float32) for tag in query_tags])
start = time.time()
distance_matrix, idx_matrix = comp_level_index.search(query_matrix, comp_k)
end = time.time()
print('over all bottom up search time', round(end-start, 3), 'sec')

print('create', query_level, 'counter')
#distance_matrix[distance_matrix > threshold] = threshold
#score_matrix = np.ones(distance_matrix.shape) - ( distance_matrix / threshold )
score_matrix = 1/(np.exp(-(threshold-distance_matrix)*sigmoid_alpha) + 1)
row_score_dicts = []
row_score_means = []
for i in np.arange(distance_matrix.shape[0]):
    row_score_dicts.append(dict())
    row_score_means.append(np.mean(score_matrix[i]))#(1 - np.mean(score_matrix[i]))
    for j in np.arange(distance_matrix.shape[1]):
        if score_matrix[i][j] <= 0:
            continue
        c = get_code(comp_level_tags[idx_matrix[i][j]], query_level)
        row_score_dicts[-1][c] = max(row_score_dicts[-1].get(c, 0.0), score_matrix[i][j])
normalized_row_score_means = normalizer(np.array(row_score_means))
score_dict = dict()
weighted_score_dict = dict()
for i in np.arange(len(row_score_dicts)):
    for k, v in row_score_dicts[i].items():
        score_dict[k] = score_dict.get(k, 0.0) + v/len(row_score_dicts)
        weighted_score_dict[k] = weighted_score_dict.get(k, 0.0) + v*normalized_row_score_means[i]
    """
    qlevel_counter.update(Counter(
        np.unique(np.array([get_code(comp_level_tags[tidx], query_level) for tidx in idx_list]))
    ))
    """

        
target_law_tags = [query]
result_df[comp_k] = pd.DataFrame(columns=['tag', 'name', 'score', 'wscore', 'ws-s','distance', 'top_down_rank'])
rank = 0
for tag, count in sorted(score_dict.items(), key=lambda x: -x[1])[:100]:
    if tag == query:
        continue
    ldist = None
    qs_rank = None
    for r, (ltidx, dist) in enumerate(qsearch_result):
        if query_level_tags[ltidx] == tag:
            ldist = dist
            qs_rank = r
            break
    target_law_tags.append(tag)
    rank += 1
    result_df[comp_k].loc[rank] = [tag, testset.get_elem(tag), count, weighted_score_dict.get(tag, 0), weighted_score_dict.get(tag, 0) - count,ldist, qs_rank]
result_df[comp_k]#.sort_values('wscore', ascending=False)

(1, 300)
search 1000 Laws from 23 Laws
over all top search time 0.022 sec
over all bottom up search time 0.48 sec
create Law counter


,tag,name,score,wscore,ws-s,distance,top_down_rank
1,24/245437/0424/Law(1),紀北町多目的会館条例,0.527842,0.483099,-0.044744,None,None
2,21/212202/0227/Law(1),下呂市下呂温泉乗政キャンプ場設置条例,0.383188,0.520299,0.137112,0.139189,272
3,21/212202/0087/Law(1),下呂市下呂温泉乗政キャンプ場設置条例,0.383188,0.520299,0.137112,0.139189,273
4,24/245437/0589/Law(1),紀北町地域振興会館条例,0.362795,0.215450,-0.147345,None,None
5,21/213616/0225/Law(1),垂井町エコパークの設置及び管理に関する条例,0.347919,0.536127,0.188208,None,None
6,24/245437/0388/Law(1),観光サービスセンター条例,0.347552,0.197478,-0.150075,None,None
7,21/212016/0795/Law(1),岐阜市長良川鵜飼伝承館条例,0.340109,0.513582,0.173474,0.105821,84
8,24/242012/0440/Law(1),津市青山高原保健休養地の設置及び管理に関する条例,0.339932,0.537109,0.197176,None,None
9,21/212024/0699/Law(1),大垣市一之瀬ポケットパーク設置条例,0.339733,0.525713,0.185981,0.133198,218
10,24/245437/0299/Law(1),紀北町老人憩の家条例,0.335935,0.175362,-0.160573,None,None


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [264]:
1/(np.exp(-(threshold-distance_matrix)*100) + 1)

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        9.65716608e-05, 9.65716608e-05, 9.65716608e-05],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.57407296e-08, 1.57407296e-08, 1.57407296e-08],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        9.91520464e-01, 9.91520464e-01, 9.91520464e-01],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.26340174e-20, 1.26340174e-20, 1.26242374e-20],
       [1.00000000e+00, 9.99996305e-01, 9.99976873e-01, ...,
        1.20761668e-29, 1.20112035e-29, 1.20056153e-29],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        3.18318966e-13, 3.18318966e-13, 3.18318966e-13]], dtype=float32)

In [238]:
list(zip(normalized_row_score_means, row_score_means))

[(0.02515456330035066, 1.6957464826651447),
 (0.01822286873566335, 1.2284596315030833),
 (0.033030274866200596, 2.2266724234899278),
 (0.04908712207252603, 3.30911388143559),
 (0.03246816260870343, 2.1887787072630793),
 (0.0538345149516962, 3.629150237485951),
 (0.042063494591814944, 2.8356295496364345),
 (0.06831344716577671, 4.605219592441988),
 (0.038794050292872466, 2.6152262532642006),
 (0.0622395881212437, 4.195762072230252),
 (0.06517201375226107, 4.3934459035931885),
 (0.0476291896660123, 3.210830173593414),
 (0.035604002804438616, 2.400175340097639),
 (0.03470055154726697, 2.3392709120097597),
 (0.047732231275319364, 3.217776525412933),
 (0.0375387510525092, 2.5306026704127915),
 (0.028571108448913144, 1.9260662997641256),
 (0.039449313131598746, 2.6593995366849374),
 (0.06831346484918428, 4.605220784534883),
 (0.058129279152556654, 3.9186735021328776),
 (0.04043169620607876, 2.725625001356631),
 (0.049982379329313535, 3.3694659267544083),
 (0.02353793207769922, 1.586764399500

In [271]:
df = result_df[comp_k].sort_values('wscore', ascending=False).reset_index().rename(columns={'index': 'score_rank'}).reset_index().rename(columns={'index': 'wscore_rank'})

In [272]:
df['wsr-sr'] = df.apply(lambda x: x['wscore_rank']-x['score_rank'], axis=1)
df.sort_values('score_rank')

,wscore_rank,score_rank,tag,name,score,wscore,ws-s,distance,top_down_rank,wsr-sr
67,67,1,24/245437/0424/Law(1),紀北町多目的会館条例,0.610533,0.571642,-0.038892,None,None,66
87,87,2,24/245437/0589/Law(1),紀北町地域振興会館条例,0.431084,0.136557,-0.294528,None,None,85
11,11,3,21/212202/0227/Law(1),下呂市下呂温泉乗政キャンプ場設置条例,0.397817,0.870424,0.472607,0.139189,272,8
10,10,4,21/212202/0087/Law(1),下呂市下呂温泉乗政キャンプ場設置条例,0.397817,0.870424,0.472607,0.139189,273,6
90,90,5,24/245437/0299/Law(1),紀北町老人憩の家条例,0.391997,0.110562,-0.281434,None,None,85
1,1,6,24/245437/0354/Law(1),紀北町営火葬場条例,0.382242,0.893529,0.511287,None,None,-5
46,46,7,24/244716/0401/Law(1),大紀町炭焼窯施設条例,0.374940,0.705151,0.330211,0.0551338,7,39
3,3,8,21/212024/0699/Law(1),大垣市一之瀬ポケットパーク設置条例,0.365746,0.884353,0.518607,0.133198,218,-5
93,93,9,24/245437/0388/Law(1),観光サービスセンター条例,0.354414,0.094651,-0.259764,None,None,84
0,0,10,21/212075/0564/Law(1),美濃市中山間地域農村活性化施設の設置及び管理に関する条例,0.342645,0.909385,0.566740,None,None,-10


In [233]:
row_score_means

[1.6957464826651447,
 1.2284596315030833,
 2.2266724234899278,
 3.30911388143559,
 2.1887787072630793,
 3.629150237485951,
 2.8356295496364345,
 4.605219592441988,
 2.6152262532642006,
 4.195762072230252,
 4.3934459035931885,
 3.210830173593414,
 2.400175340097639,
 2.3392709120097597,
 3.217776525412933,
 2.5306026704127915,
 1.9260662997641256,
 2.6593995366849374,
 4.605220784534883,
 3.9186735021328776,
 2.725625001356631,
 3.3694659267544083,
 1.5867643995001544]

In [274]:
target_law_tags = [query, '21/212202/0227/Law(1)']
df = hmodel.comptable(testset, query, target_law_tags[1:], 10, 'Sentence')
df['weight'] = normalized_row_score_means
display(df.sort_values('weight').style.apply(lambda x: coloring(0.2, x, [0], [3]), axis=1))

In [199]:
for r, (ltidx, dist) in enumerate(qsearch_result):
    print(r, query_level_tags[ltidx], testset.get_elem(query_level_tags[ltidx]), dist)

0 24/245437/0420/Law(1) 紀北町農林業試作園条例 0.0
1 21/214043/0195/Law(1) 池田町農産物加工所の設置及び管理に関する条例 0.04855535
2 22/222194/0551/Law(1) 下田市農林水産物処理加工施設条例 0.048992142
3 24/244716/0363/Law(1) 大紀町フラワーガーデンＫＩＳＥＩ条例 0.052601893
4 21/212202/1178/Law(1) 舞台峠ドームの設置及び管理に関する条例 0.052682072
5 21/212032/0459/Law(1) 飛騨高山ビッグアリーナの設置及び管理に関する条例 0.053605594
6 23/232068/0249/Law(1) 春日井市野外キャンプ場の設置及び管理に関する条例 0.05382947
7 24/244716/0401/Law(1) 大紀町炭焼窯施設条例 0.055133764
8 24/244716/0395/Law(1) 大紀町交流施設もみじ茶屋条例 0.059258766
9 21/212121/0276/Law(1) 土岐市牧草地の設置及び管理に関する条例 0.060735114
10 21/212202/0920/Law(1) 下呂市肥育牛舎の設置及び管理に関する条例 0.06582402
11 24/245437/0394/Law(1) 紀北町種まき権兵衛の里条例 0.06866373
12 21/213616/0389/Law(1) 垂井町弓道場設置及び管理に関する条例 0.07022702
13 21/212032/0670/Law(1) 高山市飛騨プラネタリウムの設置及び管理に関する条例 0.070373245
14 21/212113/0662/Law(1) 美濃加茂市あじさいエコパークの設置及び管理に関する条例 0.07045944
15 24/244716/0397/Law(1) 大紀町菌床椎茸培養・栽培施設条例 0.071467325
16 22/222267/0224/Law(1) 牧之原市相良シーサイドパーク管理規則 0.073143184
17 22/224294/0445/Law(1) 川根本町自然休養村農林水産物直売所条例 0.07398717
18 22/22

238 21/214043/0196/Law(1) 池田町農産物加工所の設置及び管理に関する条例施行規則 0.13589558
239 21/214035/0196/Law(1) 大野町農村集落多目的施設の管理及び運営に関する規則 0.13603207
240 23/235636/0022/Law(1) 豊根村患者輸送車の管理に関する規則 0.1360606
241 21/214043/0193/Law(1) 池田町転作定着化研修施設設置規則 0.13620299
242 23/231002/0411/Law(1) 名古屋市公設市場条例 0.13621753
243 23/232327/0244/Law(1) 愛西市霊園の設置及び管理に関する条例 0.13646245
244 21/212202/0983/Law(1) 下呂市老谷自然の森条例 0.13649994
245 21/212202/0778/Law(1) 下呂市老谷自然の森条例 0.13649994
246 23/234273/0139/Law(1) 飛島村公民館分館の設置及び管理に関する条例 0.13650379
247 21/215066/0274/Law(1) 白川町地域資源活用総合交流促進施設の管理及び運営に関する規則 0.13659416
248 23/232157/0506/Law(1) 旧磯部家住宅復原施設の設置及び管理に関する条例 0.13660112
249 24/242012/0388/Law(1) 津市営墓園の設置及び管理に関する条例 0.13667423
250 22/224294/0357/Law(1) フォーレなかかわね茶茗舘条例施行規則 0.1366974
251 24/242021/0472/Law(1) 四日市市大型共同作業場条例 0.13722104
252 23/232211/0417/Law(1) 新城市生涯学習センターちさと館の設置及び管理に関する条例 0.13722193
253 21/212121/0434/Law(1) 土岐市射撃場の設置及び管理に関する条例施行規則 0.137334
254 22/223069/0419/Law(1) 西伊豆町消防団分団詰所条例 0.13737252
255 24/244716/0399/Law(1) 大紀町陶芸小屋条例 0

458 23/232106/0389/Law(1) フローラルガーデンよさみ条例施行規則 0.1639514
459 22/221309/0297/Law(1) 浜松市と畜場条例 0.16400021
460 22/222089/0593/Law(1) 伊東市大原武道場条例 0.16410324
461 21/212199/0418/Law(1) 郡上市白鳥地域資源活用交流施設油坂さくらパークの設置及び管理に関する条例施行規則 0.16413121
462 22/223018/0412/Law(1) 東伊豆町海水浴場管理運営規則 0.16458297
463 22/222101/0449/Law(1) 富士市休養林条例施行規則 0.16477922
464 22/222071/0438/Law(1) 富士宮市田貫湖キヤンプ場施設条例施行規則 0.16487932
465 23/232033/0724/Law(1) アイプラザ一宮の設置及び管理に関する条例 0.164949
466 23/232211/0444/Law(1) 新城市リフレッシュセンターの設置及び管理に関する条例 0.16497284
467 21/212059/0659/Law(1) 関市武芸川ふるさと館条例 0.16503087
468 22/223026/0415/Law(1) 河津町海水浴場管理運営規則 0.16513652
469 23/232157/0291/Law(1) 犬山城入場登閣料等徴収条例 0.16520955
470 22/223026/0278/Law(1) 河津町Ｂ＆Ｇ海洋センターの管理及び運営に関する条例 0.16526076
471 24/244716/0485/Law(1) 大紀町コンベンションホール管理規則 0.16528706
472 23/232271/0257/Law(1) 高浜市心身障害児福祉施設みどり学園の設置及び管理に関する条例 0.16555694
473 21/212067/0440/Law(1) 中津川市フィールド施設の設置等に関する条例 0.16559595
474 22/222038/0739/Law(1) 沼津市渡し船条例施行規則 0.16571273
475 22/223069/0574/Law(1) 西伊豆町夕陽ビュースポット条例 0.16

624 21/216046/0461/Law(1) 白川村村民グラウンド設置条例施行規則 0.18182904
625 23/235610/0351/Law(1) 設楽町歴史の里田峯城条例 0.18195282
626 21/212075/0320/Law(1) 美濃市生涯学習施設の設置及び管理に関する条例施行規則 0.18204477
627 21/212024/0719/Law(1) 大垣市時山簡易宿泊施設設置条例施行規則 0.18205377
628 24/242039/0216/Law(1) 山田奉行所記念館条例 0.18215151
629 23/232211/0327/Law(1) 新城市四谷千枚田多目的施設の設置及び管理に関する条例 0.18226808
630 21/212059/0667/Law(1) 関市立洞戸高賀山自然の家条例施行規則 0.18227527
631 23/232211/0456/Law(1) 鳳来寺山歴史文化考証館等の設置及び管理に関する条例 0.18233152
632 23/231002/1041/Law(1) 名古屋市クリーニング業法施行条例 0.18233702
633 21/215023/0193/Law(1) 富加町半布里農園の設置に関する条例 0.18246527
634 24/242012/0307/Law(1) 津市久居厚生寮の設置及び管理に関する条例施行規則 0.18264309
635 23/232386/0325/Law(1) 長久手市セーフティステーション設置条例施行規則 0.18292814
636 22/222216/0660/Law(1) 湖西市レンタサイクル条例 0.18300104
637 22/222241/0417/Law(1) 菊川市畜犬管理条例 0.1830047
638 22/222143/0239/Law(1) 藤枝市民テニス場条例 0.18300967
639 22/222194/0529/Law(1) 下田市海水浴場に関する条例 0.1830324
640 22/224294/0321/Law(1) 川根本町農道管理規則 0.18316919
641 22/222038/0514/Law(1) 沼津市戸田テニスコート条例施行規則 0.18326542
642 22/222194

786 22/220001/0708/Law(1) 静岡県金属くず営業条例施行規則 0.19714692
787 21/215023/0191/Law(1) 富加町転作研修センター設置条例 0.19721225
788 22/223018/0251/Law(1) 東伊豆町総合グラウンドの設置及び管理等に関する条例 0.19722426
789 21/212105/0401/Law(1) 恵那市上矢作協業活動拠点施設条例 0.19735304
790 24/242055/0683/Law(1) 桑名市水郷テラス条例 0.19742626
791 22/223441/0276/Law(1) 水路及び土砂扞止林用地管理規則 0.19752018
792 22/223069/0215/Law(1) 西伊豆町艇庫条例 0.19754589
793 24/245437/0401/Law(1) 紀北町和具の浜海水浴場条例 0.19756693
794 21/213811/0172/Law(1) 神戸町斎苑の設置及び管理に関する条例施行規則 0.19760686
795 21/215210/0121/Law(1) 御嵩町夜間照明施設使用料徴収条例 0.19769424
796 22/222101/0448/Law(1) 富士市休養林条例 0.19781563
797 23/232025/0515/Law(1) 岡崎市動物の愛護及び管理に関する条例施行規則 0.19786178
798 23/232157/0284/Law(1) 犬山市弓道場の設置及び管理に関する条例 0.19786541
799 24/242039/0316/Law(1) 伊勢市隣保館条例施行規則 0.19786677
800 24/242128/0264/Law(1) 熊野市花木採取の規制に関する規則 0.1978854
801 23/235636/0140/Law(1) 豊根村立学校施設管理細則 0.19790162
802 22/222062/0472/Law(1) 三島市墓園条例施行規則 0.1979334
803 22/222127/0373/Law(1) 焼津市漁船員テニス場条例施行規則 0.1979959
804 22/220001/0712/Law(1) 静岡県集団示威運動等に関する条例 0.198

987 23/232084/0418/Law(1) 飼犬を伴う散歩に関する条例 0.21216476
988 21/212059/0679/Law(1) 関市ネイチャーランドかみのほ条例施行規則 0.212184
989 21/214019/0302/Law(1) 揖斐川町脛永農林水産物販売所の設置及び管理に関する条例 0.21221864
990 23/232165/0291/Law(1) 常滑市営モーターボート競走条例 0.21224459
991 22/222038/0502/Law(1) 沼津市香陵運動場管理規則 0.21231997
992 23/232262/0308/Law(1) 尾張旭市草刈機貸出規則 0.21247469
993 24/242021/1121/Law(1) 四日市市観光大使設置条例 0.2125185
994 21/212032/0655/Law(1) 高山市防災ダム等管理条例 0.21256202
995 24/242071/0336/Law(1) 鈴鹿市コミュニティセンター条例 0.21271531
996 22/221309/0908/Law(1) 浜松市納骨堂条例施行規則 0.21280295
997 24/242128/0267/Law(1) 熊野市木津呂森林総合利用施設条例 0.21280983
998 24/243035/0176/Law(1) 木曽岬町農村集落多目的共同利用施設の設置及び管理に関する条例施行規則 0.21292439
999 21/212091/0038/Law(1) 羽島市職員の職階制に関する条例 0.21303983


In [3]:
query = "23/230006/1813/Law(1)"
qlevel = re.split('\(', os.path.split(query)[1])[0]
comp_level = 'Sentence'
qk = 100000
comp_k = 100000
threshold = 0.2
print("query:", testset.get_elem(query))

query: 愛知県暴力団排除条例


In [4]:
result_df = dict()

print('Construct index for', comp_level, 'level')
testset.set_iterator_mode(level=comp_level, tag=True, sentence=False)
tags = list(testset)
qdata_matrix = np.matrix([hmodel.layers[comp_level][tag] for tag in tags])
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(qdata_matrix)
start = time.time()
index.createIndex({'post': 2}, print_progress=True)
end = time.time()
print('ANN', comp_level, 'preparation time %f' % (end - start))

print('Construct index for', qlevel, 'level')
testset.set_iterator_mode(level=qlevel, tag=True, sentence=False)
qtags = list(testset)
print('using', len(qtags), qlevel+'s')
start = time.time()
cdata_matrix = np.matrix([hmodel.layers[qlevel][tag] for tag in qtags])
sindx = NearestNeighbors(n_neighbors=min(qk, cdata_matrix.shape[0]), metric='cosine', algorithm='brute').fit(cdata_matrix)
end = time.time()
print('KNN', comp_level, 'preparation time %f' % (end - start))

Construct index for Sentence level
ANN Sentence preparation time 2.016156
Construct index for Law level
using 688 Laws
KNN Sentence preparation time 0.008544


In [5]:
# 条文対応表作成特化型検索
start = time.time() 
d, i = sindx.kneighbors(np.matrix(hmodel.layers[qlevel][query]))
end = time.time()
qsearch_result =  list(zip(i[0], d[0]))
print('search', len(qsearch_result), qlevel+'s', 'from', len(qtags), qlevel+'s')
print('over all top search time', round(end-start, 3), 'sec')

"""
print('Construct index for', qlevel, 'level')
testset.set_iterator_mode(level=qlevel, tag=True, sentence=False)
qtags = list(testset)
print('using', len(qtags), qlevel+'s')
vectors = np.matrix([self.layers[level][tag] for tag in tags])
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(vectors)
index.createIndex({'post': 2}, print_progress=True)
print(vectors.shape)
"""


print('create', qlevel, 'counter')
qlevel_counter = Counter()
search_time = 0.0
for elem in testset.get_elem(query).depth_first_iteration([getattr(etypes, comp_level)]):
    t = time.time()
    tidx_list, distance_list = index.knnQuery(hmodel.layers[comp_level][elem.code], k=comp_k+1)
    search_time += time.time() - t
    for tidx, distance in zip(tidx_list, distance_list):
        if distance > threshold:
            break
        i = 0
        tag_parts = Path(tags[tidx]).parts
        for p in tag_parts:
            if qlevel in p:
                break
            i += 1
        qlevel_tag = '/'.join(tag_parts[:i+1])
        qlevel_counter[qlevel_tag] = qlevel_counter.get(qlevel_tag, 0) + 1
print('over all bottom up search time', round(search_time, 3), 'sec')
        
target_law_tags = [query]
result_df[comp_k] = pd.DataFrame(columns=['tag', 'name', 'similar_count', 'distance', 'top_down_rank'])
rank = 0
for tag, count in qlevel_counter.most_common(100+1):
    if tag == query:
        continue
    ldist = 'Out of top{}'.format(qk)
    qs_rank = None
    for r, (ltidx, dist) in enumerate(qsearch_result):
        if qtags[ltidx] == tag:
            ldist = dist
            qs_rank = r
            break
    target_law_tags.append(tag)
    rank += 1
    result_df[comp_k].loc[rank] = [tag, testset.get_elem(tag), count, ldist, qs_rank]
result_df[comp_k]

search 688 Laws from 688 Laws
over all top search time 0.011 sec
create Law counter
over all bottom up search time 0.023 sec


,tag,name,similar_count,distance,top_down_rank
1,23/230006/1874/Law(1),薬物の濫用の防止に関する条例,24,0.838424,100
2,23/230006/1808/Law(1),愛知県風俗案内所規制条例,23,0.682834,1
3,23/230006/1815/Law(1),公衆に著しく迷惑をかける暴力的不良行為等の防止に関する条例,23,0.782432,14
4,23/230006/0934/Law(1),愛知県漁港管理条例,14,0.830802,79
5,23/230006/0678/Law(1),愛知県立自然公園条例,13,0.747326,5
6,23/230006/0991/Law(1),愛知県屋外広告物条例,11,0.831380,81
7,23/230006/1896/Law(1),公衆に著しく迷惑をかける暴力的不良行為等の防止に関する条例施行規則,9,0.852533,136
8,23/230006/1832/Law(1),暴走族等の追放の促進に関する条例,7,0.829379,72
9,23/230006/1806/Law(1),風俗営業等の規制及び業務の適正化等に関する法律施行条例,7,0.795525,28
10,23/230006/0679/Law(1),愛知県立自然公園条例施行規則,6,0.721639,4


In [32]:
target_law_tags = ['23/230006/1813/Law(1)', '22/222267/0359/Law(1)']
query = target_law_tags[0]

In [33]:
target_law_tags = ['23/230006/1813/Law(1)', '22/222267/0359/Law(1)']
df = hmodel.comptable(testset, query, target_law_tags[1:], 10, 'Sentence')
display(df.style.apply(lambda x: coloring(threshold, x), axis=1))

In [18]:
def topk_comptable(self, dataset, query, k, threshold, level, root_threshold=None):
    qlevel = re.split('\(', os.path.split(query)[1])[0]
    dataset.set_iterator_mode(level=qlevel, tag=True, sentence=False)
    law_tags = list(dataset)
    _, law_index = self.create_index(qlevel, law_tags)
    tidx, similarity = law_index.knnQuery(self.layers[qlevel][query], k=k+1)
    target_law_tags = []
    sims = []
    for i, sim in zip(tidx, similarity):
        if law_tags[i] == query:
            continue
        if root_threshold and sim < root_threshold:
            print("not enough distance", dataset.get_elem(law_tags[i]))
            k -= 1
            continue
        target_law_tags.append(law_tags[i])
        sims.append(sim)
    print(query)
    for i, t in enumerate(target_law_tags):
        print(i, dataset.get_elem(t), sims[i])
    #return self.comptable(dataset, query, target_law_tags[:k], threshold, level)
df = topk_comptable(hmodel, dataset=testset, query=query, threshold=10, level='Sentence', k=100)
display(df.style.apply(lambda x: coloring(threshold, x), axis=1))

23/230006/1813/Law(1)
0 薬物の濫用の防止に関する条例 0.22951794
1 暴走族等の追放の促進に関する条例 0.24275112
2 県民の消費生活の安定及び向上に関する条例 0.26631784
3 拡声機による暴騒音の規制に関する条例 0.28003824
4 動物の愛護及び管理に関する条例 0.28233266
5 愛知県風俗案内所規制条例 0.30496156
6 風俗営業等の規制及び業務の適正化等に関する法律施行条例 0.32943392
7 公衆に著しく迷惑をかける暴力的不良行為等の防止に関する条例 0.3688966
8 食と緑が支える県民の豊かな暮らしづくり条例 0.38045692
9 警備業者及び警備員の護身用具の携帯の禁止及び制限に関する規則 0.38056177
10 薬物の濫用の防止に関する条例施行規則 0.39303815
11 中小企業者等向け融資の損失補償に係る回収納付金を受け取る権利の放棄に関する条例 0.39744544
12 愛知県警察教養規則 0.4014014
13 空き缶等ごみの散乱の防止に関する条例 0.4075964
14 愛知県統計調査条例 0.41006756
15 愛知県県民生活安定緊急対策条例 0.4222839
16 愛知県地震防災推進条例 0.4356575
17 愛知県安全なまちづくり条例施行規則 0.4401182
18 愛知県家畜伝染病まん延防止規則 0.4490279
19 県民の消費生活の安定及び向上に関する条例施行規則 0.45877504
20 愛知県観光振興基本条例 0.46062034
21 知事の保有する個人情報の保護等に関する規則 0.46330148
22 理容業に係る衛生措置等に関する条例 0.4769199
23 愛知県公害健康被害認定審査会条例 0.48139662
24 愛知県情報公開条例 0.4823928
25 公衆に著しく迷惑をかける暴力的不良行為等の防止に関する条例施行規則 0.48260772
26 愛知県ふれあい広場規則 0.48422474
27 旅館業法施行条例 0.4847405
28 あいち海上の森条例 0.48631412
29 蜜蜂の腐蛆病のまん延防止に関する規則 0.48875546
30 愛知県ふぐ取扱い規制条例 0.

AttributeError: 'NoneType' object has no attribute 'style'

In [8]:
columns = []
for k, df in dfdict.items():
    q = (target_law_tags[0], 'Sentence')
    df.columns = pd.MultiIndex.from_tuples([q, (k, 'Sentence'), (k, 'score')])
    print(k)
    if 'spm' in k:
        from hjst_model import tokenizer
        tk = lambda x: '/'.join([w for w in tokenizer.load_spm(test_sentence_layers[k]['layer'].spm_path).EncodeAsPieces(x)])
        df[(k, 'Sentence')] = df[(k, 'Sentence')].map(tk)


swem_average_tokai_mecab
swem_ave_ft_tokai_spm_8000
swem_ave_gv_tokai_spm_8000
swem_average_tokai_spm_8000
swem_ave_ft_tokai_mecab


In [9]:
pd.merge(
    dfdict['swem_average_tokai_spm_8000'],
    dfdict['swem_ave_gv_tokai_spm_8000']
).style.apply(lambda x: coloring(0.2, x), axis=1)

In [1]:
for k, layer in test_sentence_layers.keys():
    print(k)
    pd.DataFrame([pd.DataFrame(pd.Series(arr.ravel()).describe()).transpose() for arr in layer.vecs])

NameError: name 'test_sentence_layers' is not defined

In [34]:
dfdict['swem_average_tokai_mecab']["名古屋市暴力団排除条例(23/231002/0999/Law(1))"]

,Sentence,Distance
1,この条例は、市における暴力団の排除の推進に関し、基本理念を定め、並びに市、市民及び事業者の責務を明らかにするとともに、市の施策の基本となる事項等を定めることにより、市、市民及び事業者が一体となって暴力団の排除を推進し、もって市民の安全で平穏な生活を確保し、及び社会経済活動の健全な発展に寄与することを目的とする。,0.055
2,この条例において、次の各号に掲げる用語の意義は、それぞれ当該各号に定めるところによる。,0
3,暴力団の排除は、暴力団が市民の生活、市内の事業活動及び青少年の健全な育成に不当な影響を及ぼす存在であることを認識した上で、暴力団を恐れないこと、暴力団を利用しないこと、暴力団に協力しないこと及び暴力団と交際しないことを基本として推進されなければならない。,0.012
4,暴力団の排除は、市、市民及び事業者が相互に連携し、及び協力して推進するものとする。,0.088
5,市は、前条に定める基本理念(以下「基本理念」という。)にのっとり、市民及び事業者の協力を得るとともに、愛知県(以下「県」という。)及び法第32条の2第1項の規定により愛知県暴力追放運動推進センターとして指定を受けた者その他の暴力団員による不当な行為の防止を目的とする団体(以下「推進センター等」という。)と連携を図りながら、暴力団の排除に関する施策を実施するものとする。,0.039
6,市は、県及び推進センター等と連携し、市民及び事業者が暴力団の排除に自主的に、かつ、相互に連携して取り組むことができるよう、市民及び事業者に対し、情報の提供その他の必要な支援を行うものとする。,0.169
7,事業者は、基本理念にのっとり、その行う事業により暴力団を利することとならないようにするとともに、市が実施する暴力団の排除に関する施策に協力しなければならない。,0.005
8,市民及び事業者は、暴力団の排除に資すると認められる情報を知ったときは、市、愛知県警察その他の関係行政機関に対し、当該情報を提供するよう努めなければならない。,0.082
9,市民は、基本理念にのっとり、暴力団の排除に自主的に、かつ、相互に連携して取り組むよう努めるとともに、市が実施する暴力団の排除に関する施策に協力するよう努めなければならない。,0.006
10,市民及び事業者は、暴力団の排除に資すると認められる情報を知ったときは、市、愛知県警察その他の関係行政機関に対し、当該情報を提供するよう努めなければならない。,0.092


In [22]:
threshold = 0.3
df = hmodel.comptable(testset, query, target_law_tags[1:3], 10, 'Sentence')
#display(df.style.apply(lambda x: coloring(d['threshold'], x), axis=1))
df

愛知県暴力団排除条例(23/230006/1813/Law(1))  \
                                                                                                                                                                                                                                                                                   Sentence   
1                                                                                                                         この条例は、暴力団の排除について、基本理念を定め、並びに県、事業者及び県民の責務を明らかにするとともに、県が実施する施策の基本となる事項等を定め、及び必要な規制を行うことにより、県、事業者及び県民が一体となって暴力団の排除を推進し、もって地域経済の健全な発展に寄与し、及び県民の安全で平穏な生活を確保することを目的とする。   
2                                                                                                                                                                                                                                               この条例において、次の各号に掲げる用語の意義は、それぞれ当該各号に定めるところによる。   
3                                                                                                                                                                                暴力団の排除は、暴力団が県内の事業活動及び県民の生活に不当な影響を与える存在であることを認識した上で、暴力団を利用しないこと、暴力団に協力しないこと及び暴力団と交際しないことを基本として推進されなければならない。   
4                                                                                                                                                                                                                                                          暴力団の排除は、県、事業者及び県民が協働して推進するものとする。   
5                                                                                                                                 県は、前条に定める基本理念(以下「基本理念」という。)にのっとり、法第三十二条の三第一項の規定により愛知県暴力追放運動推進センターとして指定を受けた者その他の暴力団員による不当な行為の防止を目的とする団体(以下「推進センター等」という。)と連携を図りながら、暴力団の排除に関する施策を実施する責務を有する。   
6                                                                                                                                                                                                                           県は、市町村が暴力団の排除に関する施策を実施する場合には、情報の提供、技術的な助言その他の必要な協力及び支援を行うものとする。   
7                                                                                                                                                                                                           事業者は、基本理念にのっとり、その行う事業により暴力団を利することとならないようにするとともに、県が実施する暴力団の排除に関する施策に協力しなければならない。   
8                                                                                                                                                                                                                                事業者は、暴力団の排除に資すると認められる情報を知ったときは、県に対し、当該情報を提供するよう努めなければならない。   
9                                                                                                                                                                                                    県民は、基本理念にのっとり、暴力団の排除に自主的に、かつ、相互に連携して取り組むよう努めるとともに、県が実施する暴力団の排除に関する施策に協力するよう努めなければならない。   
10                                                                                                                                                                                                                                県民は、暴力団の排除に資すると認められる情報を知ったときは、県に対し、当該情報を提供するよう努めなければならない。   
11                                                                                                                                                                                                                                              この条例の適用に当たっては、県民の権利を不当に侵害しないように留意しなければならない。   
12                                                                                                                                                                  県は、公共工事その他の県の事務又は事業により暴力団を利することとならないように、暴力団員又は暴力団若しくは暴力団員と密接な関係を有する者を公共工事の入札に参加させないことその他の暴力団の排除のために必要な措置を講ずるよう努めるものとする。   
13                                                                                              知事若しくは教育委員会又は指定管理者(地方自治法(昭和二十二年法律第六十七号)第二百四十四条の二第三項に規定する指定管理者をいう。)は、公の施設の利用の許可の申請があった場合において、当該公の施設が暴力団の活動に利用されることにより当該暴力団の利益になると認めるときは、当

In [10]:
query = next(ltag_gen)
k = 1
threshold = 0.3

print("query:", str(testset.get_elem(law_tags[i])), get_caption(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
pd.DataFrame(
    [( str(testset.get_elem(law_tags[i])), get_caption(law_tags[i]), round(s, 3), testset.get_elem(law_tags[i]).text) for i, s in zip(*list( law_index.knnQuery(hmodel.layers[qlevel][query], 20) ))],
    columns = ["name", "caption", "similarity", "text"]
)

NameError: name 'i' is not defined

In [11]:
k = 1
root_threshold =  1000
threshold = 0.3
print("query:", testset.get_elem(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
tidx, distance = law_index.knnQuery(hmodel.layers[qlevel][query], k=k+1)
target_law_tags = [query]
for i, dist in zip(tidx, distance):
    if law_tags[i] == query:
        continue
    if root_threshold and dist > root_threshold:
        print("not enough distance", testset.get_elem(law_tags[i]), sim)
        k -= 1
        continue
    print("add target:", testset.get_elem(law_tags[i]), sim)
    target_law_tags.append(law_tags[i])
for qi, q in enumerate(target_law_tags):
    df = hmodel.comptable(testset, q, target_law_tags[:qi]+target_law_tags[qi+1:], 10, 'Sentence')
    display(df.style.apply(lambda x: coloring(threshold, x), axis=1))

query: 愛知県公告式条例


NameError: name 'sim' is not defined

query: 議決事件指定条例
特別職の指定に関する条例 5.9604645e-08
議決事件指定条例 1.1920929e-07
沿道区域指定の基準に関する条例 1.1920929e-07
都市的施設その他の都市としての要件に関する条例 0.009170651
豊橋市新型インフルエンザ等対策本部条例 0.23891258
西尾市新型インフルエンザ等対策本部条例 0.23891258
職員の退職手当に関する条例の一部を改正する条例附則第二項の規定により退職手当の支給を受ける職員の範囲を定める規則 0.25446343
公立学校職員の退職手当に関する条例の一部を改正する条例附則第二項の規定により退職手当の支給を受ける職員の範囲を定める規則 0.26839417
南知多町における高齢者、障害者等の移動等の円滑化のために必要な特定公園施設の設置の基準に関する条例 0.31211913
武豊町移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.31211913
尾張旭市指定地域密着型サービス事業者及び指定地域密着型介護予防サービス事業者の指定に関する基準を定める条例 0.3121192
尾張旭市道路構造の技術的基準を定める条例 0.3121192
尾張旭市市道に設ける案内標識等の寸法を定める条例 0.3121192
尾張旭市準用河川管理施設等の構造の技術的基準を定める条例 0.3121192
尾張旭市移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.3121192
尾張旭市水道事業の布設工事監督者及び水道技術管理者に関する基準を定める条例 0.3121192
東郷町不育症治療費の助成に関する条例 0.3121192
東郷町移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.3121192
南知多町水道事業の布設工事監督者及び水道技術管理者に関する条例 0.3121192
西尾市教育委員会公告式規則 0.32603073


In [5]:
with open("./../reikiset/21/210005/0067.xml") as f:
    print(f.read())

﻿<?xml version="1.0" encoding="UTF-8"?>
<Laws DictVer="">
<Law OriginalPromulgateDate="昭和二十四年三月二十八日条例第十四号" LawType="" Lang="ja" Year="" Era="" Num="">
	<LawNum>昭和二十四年三月二十八日条例第十四号</LawNum>
	<LawBody>
		<LawTitle>岐阜県公衆浴場法施行条例</LawTitle>
		<EnactStatement>昭和二十三年七月法律第百三十九号公衆浴場法により、岐阜県公衆浴場法施行条例を、次のように定める。</EnactStatement>
		<MainProvision>
			<Article Num="1">
				<ArticleTitle>第一条</ArticleTitle>
				<Paragraph Num="1">
					<ParagraphNum />
					<ParagraphSentence>
						<Sentence>公衆浴場法第二条第三項の規定による公衆浴場の設置の場所の配置の基準並びに同法第三条第二項の規定による公衆浴場の換気、採光、照明保温及び清潔その他入浴者の衛生及び風紀について必要な措置の基準は、この条例の定めるところによる。</Sentence>
					</ParagraphSentence>
				</Paragraph>
			</Article>
			<Article Num="2">
				<ArticleTitle>第二条</ArticleTitle>
				<Paragraph Num="1">
					<ParagraphNum />
					<ParagraphSentence>
						<Sentence Num="1" Function="Main">新たに設置しようとする公衆浴場は、既設の公衆浴場との間に、次に掲げる距離を保たなければならない。</Sentence>
						<Sentence Num="2" Function="Proviso">ただし、土地の状況、予想利用者数、人口密度等を考慮して公衆浴場の設置が公衆衛生上必要であると知事が認めたときは、この限りでない。</

In [ ]:
nagoya = '23/231002/0999/Law(1) 名古屋市暴力団排除条例'
aichi = '23/230006/1813/Law(1) 愛知県暴力団排除条例'
seto = '23/232041/0562/Law(1) 瀬戸市暴力団排除条例'

In [24]:
import sys 
import nmslib 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [25]:
# Read data points
testset.set_iterator_mode(level='Sentence', tag=True, sentence=False)
all_data_matrix =  np.matrix([hmodel.layers["Sentence"][tag] for tag in testset])

In [39]:
all_data_tags = {i:tag for i,tag in enumerate(testset)}

In [62]:
sample_data_matrix = all_data_matrix

In [27]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(sample_data_matrix, test_size = 0.01)

In [63]:
query_matrix = np.matrix([query_vec])
data_matrix = sample_data_matrix
query_matrix.shape

(1, 500)

In [64]:
# Set index parameters
# These are the most important onese
M = 30
efC = 100

num_threads = 1
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [65]:
# Number of neighbors 
K=100

In [66]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space='cosinesimil') 
index.addDataPointBatch(data_matrix) 

610982

In [67]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 30, 'indexThreadQty': 1, 'efConstruction': 100, 'post': 0}
Indexing time = 396.422498


In [97]:
%%timeit

index.createIndex(index_time_params) 

2.67 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params) 

Setting query-time parameters {'efSearch': 100}


In [112]:
data_matrix[1].tolist()[0] == index[0]

False

In [84]:
query_text = """動物の種類・発育状況に応じて、適正に餌・水を与えること。"""
vectorizer = hmodel.layers['Sentence']
vectorizer.model.load_wvmodel('/home/jovyan/results/hjst/models/wva_tokai_spm_16000/Sentence.lmodel/model_body/wvmodel.model')
tokenizer = spm.SentencePieceProcessor()
tokenizer.load('/home/jovyan/results/hjst/models/wva_tokai_spm_16000/Sentence.lmodel/spm/model.model')
query_vec = vectorizer.model._calc_vec(tokenizer.EncodeAsPieces(query_text))
query_matrix = np.matrix([query_vec])

In [85]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 


kNN time total=0.000645 (sec), per query=0.000645 (sec), per query adjusted for thread number=0.000645 (sec)


In [86]:
for rank, (idx, sim) in enumerate(zip(nbrs[0][0], nbrs[0][1])):
    print(rank, sim)
    elem = testset.get_elem(all_data_tags[idx])
    print(elem)
    print(elem.text)

0 0.249305
岡崎市動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
動物の種類、発育状況等に応じて適正に飼料及び水を給与すること。
1 0.329451
岡崎市動物の愛護及び管理に関する条例第2章第5条第1項第3号第1文
必要に応じて動物の種類、習性及び生理を考慮した飼養施設を設け、当該施設の設置に当たっては、日照、通風等の確保を図るなど適切な飼養環境を確保すること。
2 0.348658
豊橋市動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
適正に飼料及び水を与えること。
3 0.360377
豊橋市の政策推進における部等の役割を定める条例第3条第2項第4号3号細分第1文
水源林の涵養や新規水源の開発などにより、恒久・安定的な水資源の確保を図る。
4 0.376204
豊橋市動物の愛護及び管理に関する条例第2章第5条第1項第4号第1文
汚物及び汚水を適正に処理すること等により、飼養施設の内外を常に清潔に保ち、悪臭又は昆虫等の発生を防止すること。
5 0.37912
豊橋市の政策推進における部等の役割を定める条例第3条第2項第16号2号細分第1文
学校給食・学校保健管理、食育・食農教育の推進により、子どもの心と体の健康増進を図る。
6 0.381168
一宮市環境基本条例第2章第7条第1項第2号第1文
緑地、水辺等における自然環境を地域の自然的及び社会的条件に応じて体系的に保全すること。
7 0.38126
動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
適正にえさ及び水を与えること。
8 0.38126
名古屋市動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
適正にえさ及び水を与えること。
9 0.382779
扶桑町環境基本条例第2章第8条第1項第2号第1文
生態系の多様性の確保、野生生物の種の保存等が図られるとともに、緑地、水辺等における自然環境を地域の自然的及び社会的条件に応じて体系的に保全すること。
10 0.388206
豊橋市の政策推進における部等の役割を定める条例第3条第2項第14号1号細分第1文
水資源の効率的な利用に努めるとともに、水道施設の計画的な更新と耐震化を図ることにより、安全・安心な水道水の安定供給を確保する。
11 0.39446
岡崎市動物総合センター条例第4条第1項第1

In [57]:
nbrs

[(array([192,  93, 770, 950, 194, 193, 795, 876, 791, 942, 825,  55, 235,
         928, 745, 250, 929, 934,  32, 970, 847, 964, 215,  44, 833,  42,
         901, 884, 596, 217, 237, 941, 936, 939,  56, 492, 904, 919, 935,
         952,  67,  17, 864, 948, 883, 923,  97, 896, 828, 400, 885, 790,
         867, 890, 907, 849, 827, 151, 802, 945, 498, 910,  43, 906, 499,
         933, 848, 239,  95, 870, 826, 165, 959, 186, 900, 921, 241,  99,
         889, 119, 902,   2, 220,  52, 803, 188, 195, 958, 809, 189, 310,
         857, 865, 225, 829, 149, 937, 224,  71, 145], dtype=int32),
  array([ 0.32612216,  0.33281475,  0.37241876,  0.38690841,  0.38746548,
          0.39188552,  0.39448011,  0.40212572,  0.40290123,  0.42246723,
          0.42719483,  0.43361211,  0.43824816,  0.44252771,  0.44341385,
          0.45301402,  0.45506471,  0.46236753,  0.46460223,  0.46486294,
          0.4689579 ,  0.46993113,  0.4709689 ,  0.47818649,  0.48049474,
          0.4832263 ,  0.48854315,  0.49122

In [96]:
%%timeit
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)

1.2 s ± 6.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(query_matrix)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.006200
brute-force kNN time total=1.866269 (sec), per query=0.000187 (sec)


In [94]:
%%timeit
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)

2.7 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [95]:
%%timeit
gs = sindx.kneighbors(query_matrix)

1.12 s ± 36.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.988400


In [15]:
data_matrix.shape

(100, 500)

In [18]:
%timeit np.dot(query_matrix, data_matrix.T).max()

151 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
%timeit np.argsort(np.dot(query_matrix[0], data_matrix.T))

31.8 µs ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [93]:
%%timeit
for q in query_matrix:
    arr = np.dot(q, data_matrix.T)
    arr.max()
    arr.max()

11.2 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
74.5 ms

0.033

In [23]:
30.6 ms

0.109923

In [24]:
183.5 ms

0.045714000000000005

In [ ]:
61.5 ms